In [1]:
import os
import shutil
from sklearn.model_selection import train_test_split

# Шлях до папки з усіма зображеннями та CSV-файлом із мітками
all_images_dir = '/kaggle/input/ukraine-ml-bootcamp-2023/images/train_images/'
csv_file_path = '/kaggle/input/ukraine-ml-bootcamp-2023/train.csv'

# Завантаження даних з CSV файлу
import pandas as pd
train_data = pd.read_csv(csv_file_path)

# Шляхи до папок для тренувальних та валідаційних зображень
train_images_dir = 'kaggle/input/ukraine-ml-bootcamp-2023/train/images'
# validation_images_dir = 'kaggle/input/ukraine-ml-bootcamp-2023/validation/images'

# Створення папок для тренувальних та валідаційних зображень
os.makedirs(train_images_dir, exist_ok=True)
# os.makedirs(validation_images_dir, exist_ok=True)

# # Розділення на тренувальний та валідаційний набори
# train_data, validation_data = train_test_split(data, test_size=0.2, random_state=42)

# Копіювання зображень до відповідних папок з розділенням за класами
for index, row in train_data.iterrows():
    class_folder = os.path.join(train_images_dir, str(row['class_6']))
    os.makedirs(class_folder, exist_ok=True)
    src_path = os.path.join(all_images_dir, row['image_id'])
    dst_path = os.path.join(class_folder, row['image_id'])
    shutil.copy(src_path, dst_path)

# for index, row in validation_data.iterrows():
#      class_folder = os.path.join(validation_images_dir, str(row['class_6']))
#      os.makedirs(class_folder, exist_ok=True)
#      src_path = os.path.join(all_images_dir, row['image_id'])
#      dst_path = os.path.join(class_folder, row['image_id'])
#      shutil.copy(src_path, dst_path)


In [2]:
from keras.applications import MobileNetV2
from keras.models import Sequential
from keras.layers import GlobalAveragePooling2D, Dense
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator

base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

model = Sequential()
model.add(base_model)
model.add(GlobalAveragePooling2D())
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(6, activation='softmax'))  # 6 класів

# Розмороження деяких шарів базової моделі
for layer in base_model.layers[10:]: 
    layer.trainable = True

# Компіляція моделі
model.compile(optimizer=Adam(lr=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# Підготовка даних та генератори
train_datagen = ImageDataGenerator(rescale=1.0/255,
                                   rotation_range=10,  # Зменшено аугментацію
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   fill_mode='nearest')

train_generator = train_datagen.flow_from_directory('kaggle/input/ukraine-ml-bootcamp-2023/train/images',
                                                    target_size=(224, 224),
                                                    batch_size=32,
                                                    class_mode='categorical')

# validation_datagen = ImageDataGenerator(rescale=1.0/255)

# validation_generator = validation_datagen.flow_from_directory('kaggle/input/ukraine-ml-bootcamp-2023/validation/images',
#                                                               target_size=(224, 224),
#                                                               batch_size=32,
#                                                               class_mode='categorical')

# Навчання моделі
model.fit(train_generator,
          steps_per_epoch=len(train_generator),
          epochs=30)

9406464/9406464 [==============================] - 0s 0us/step
Found 2360 images belonging to 6 classes.


/opt/conda/lib/python3.10/site-packages/keras/optimizers/legacy/adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


Epoch 1/30
74/74 [==============================] - 347s 5s/step - loss: 1.5735 - accuracy: 0.3737
Epoch 2/30
74/74 [==============================] - 338s 5s/step - loss: 0.9958 - accuracy: 0.6318
Epoch 3/30
74/74 [==============================] - 339s 5s/step - loss: 0.6498 - accuracy: 0.7746
Epoch 4/30
74/74 [==============================] - 338s 5s/step - loss: 0.4513 - accuracy: 0.8487
Epoch 5/30
74/74 [==============================] - 339s 5s/step - loss: 0.3172 - accuracy: 0.8979
Epoch 6/30
74/74 [==============================] - 339s 5s/step - loss: 0.2635 - accuracy: 0.9085
Epoch 7/30
74/74 [==============================] - 341s 5s/step - loss: 0.2025 - accuracy: 0.9309
Epoch 8/30
74/74 [==============================] - 340s 5s/step - loss: 0.1416 - accuracy: 0.9559
Epoch 9/30
74/74 [==============================] - 344s 5s/step - loss: 0.1100 - accuracy: 0.9699
Epoch 10/30
74/74 [==============================] - 341s 5s/step - loss: 0.0924 - accuracy: 0.9703
Epoch 11/